# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/Janus-Pro-7B',
 '/ValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-V3',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/deepseek-ai/deepseek-vl2-small',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/ReverseImageSearch/Reverse-Image-Search2',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/simplescaling/s1K',
 '/datasets/ServiceNow-AI/R1-Distill-SFT',
 '/datasets/HuggingFaceFW/fineweb',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
#print(get_all_details("https://huggingface.co"))
url = "https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/?srsltid=AfmBOoq4gB_CUiYOy3CGI7sQMTB3fYBEK-DIAIP6F5IdrYgAWm_gRjOy&couponCode=KEEPLEARNING"
print(get_all_details(url))

Found links: {'links': []}
Landing page:
Webpage Title:
Just a moment...
Webpage Contents:
Enable JavaScript and cookies to continue




In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += "Include a section on how and when Langchain is used in the course"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
#get_brochure_user_prompt("HuggingFace", "https://huggingface.co")
get_brochure_user_prompt("AI Engineering", url)

Found links: {'links': []}


'You are looking at a company called: AI Engineering\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nInclude a section on how and when Langchain is used in the courseLanding page:\nWebpage Title:\nJust a moment...\nWebpage Contents:\nEnable JavaScript and cookies to continue\n\n'

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
#create_brochure("HuggingFace", "https://huggingface.co")
create_brochure("AI Engineering", url)

Found links: {'links': []}


# AI Engineering Brochure

## Welcome to AI Engineering

At AI Engineering, we are pioneers in developing robust AI solutions that empower businesses across various sectors to harness the power of artificial intelligence. We pride ourselves on being at the forefront of innovation, creating AI systems that drive efficiency and productivity.

---

## Our Culture

At AI Engineering, our company culture is built on collaboration, creativity, and continuous learning. We foster an environment where every team member's voice is heard, and ideas can flourish. We believe in supporting our employees' professional growth through training programs and mentorship. Our diverse workforce is united by a shared passion for technology and a commitment to excellence.

- **Collaboration:** We work together across teams to share insights and drive projects forward.
- **Innovation:** Encouraging out-of-the-box thinking to create unique AI solutions.
- **Support:** Offering mentorship and tools for career development.

---

## Our Customers

AI Engineering proudly serves a diverse range of clients, including startups, small to medium-sized enterprises, and large corporations. Our solutions are tailored to meet the unique needs of each customer, ensuring high satisfaction and effective use of AI technology.

- **Industries Served:** Healthcare, Finance, Retail, Logistics, and more.
- **Client Commitment:** We prioritize customer feedback and adapt our solutions to better serve their evolving needs.

---

## Careers at AI Engineering

Join us at AI Engineering and be part of a dynamic team shaping the future of artificial intelligence. We offer a variety of positions across multiple disciplines, including:

- **Software Development**
- **Data Science**
- **Project Management**
- **Sales and Marketing**

We are looking for innovative thinkers who are excited about using AI to solve real-world problems. Our application process is straightforward, and we welcome talent from diverse backgrounds.

### Current Openings
- Junior Data Scientist
- AI Solutions Architect
- Business Development Executive

For more information on job openings and to apply, visit our careers page on the AI Engineering website.

---

## How and When to Use Langchain

### Langchain in AI Engineering

Langchain plays a crucial role in many of our AI projects, particularly in implementing natural language processing and machine learning workflows. This powerful framework helps us integrate different components of an AI system seamlessly, enabling us to deliver superior products to our clients.

- **When to Use Langchain:** Langchain is utilized during the development of applications that require advanced language models and data processing capabilities.
- **Key Applications:** Chatbots, automated customer service solutions, and sentiment analysis tools.

---

**Contact Us**  
For inquiries or further information about our services, careers, or partnerships, please visit our website or reach out to our customer service team.

**AI Engineering - Shaping the Future of AI**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [46]:
def stream_brochure(company_name, url, pretty_markdown=False):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    if pretty_markdown: 
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)
    else:
        for chunk in stream:
            print(chunk.choices[0].delta.content or '', end='')

In [48]:
stream_brochure("HuggingFace", "https://huggingface.co", pretty_markdown=True)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'social media - twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'social media - linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
At Hugging Face, we are not just a company; we are a vibrant AI community dedicated to building the future of artificial intelligence. Our innovative platform serves as a collaborative space where ML enthusiasts, researchers, and organizations can come together to share, create, and optimize models, datasets, and applications.

---

## Our Offerings
### **Models**
Explore over 1 million models ranging from text and image processing to audio and 3D generation. Some of the most popular models include:
- **DeepSeek-R1**: 2.43M updates
- **Tifa-Deepsex-14b-CoT-GGUF-Q4**: Introduced cutting-edge generative capabilities

### **Datasets**
With more than 250,000 datasets available, our platform supports access to and sharing of datasets for various machine learning tasks, helping the community accelerate research and development.

### **Spaces**
Hugging Face allows users to host and collaborate in virtual workspaces. This enables everyone from startups to large enterprises to develop applications and share their findings seamlessly.

### **Enterprise Solutions**
We provide customized enterprise-grade solutions with dedicated support, enhanced security, and flexible pricing starting at $20/user/month. More than 50,000 organizations—like Google, Amazon, and Microsoft—trust Hugging Face for their AI needs.

---

## Company Culture
At Hugging Face, our culture revolves around **collaboration**, **open-source values**, and **innovation**. We believe in building an inclusive environment where members from diverse backgrounds contribute to the collective intelligence of our community. We value transparency and encourage sharing ideas and knowledge, fostering creativity and advancement in machine learning technologies.

---

## Join Us
### **Careers**
We are constantly on the lookout for passionate individuals who want to contribute to our mission. Whether you are a researcher, developer, or any other professional in the AI field, we welcome you to be a part of our journey. Explore our [Careers Page](https://huggingface.co/jobs) to find opportunities that match your skills and interests.

---

## Who We Serve
From independent developers to large enterprises, our customers come from various sectors including technology, finance, healthcare, and education. Hugging Face is committed to empowering these organizations with the tools, models, and expertise needed to build transformative AI applications.

---

## Get In Touch
If you're interested in harnessing the power of machine learning with Hugging Face, visit us at [huggingface.co](https://huggingface.co) to explore our offerings, community forums, and resources.

---

**Hugging Face**: Together, we are building the future of AI.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>